In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.3):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 1
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 2) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
        

In [ ]:
train(npoints = 9, batchsize = 4000, nsamples = 8) #greedyprob - 3, with +ve reinforcement

14.808250427246094 -4.2974653244018555 -4.757828235626221 -4.7066755294799805 -4.706675052642822
14.144250869750977 -3.878293037414551 -4.637988567352295 -5.146239280700684 -5.146239280700684
13.945500373840332 -3.193018674850464 -4.287086486816406 -3.0437400341033936 -3.0437397956848145
13.684000968933105 -2.6916637420654297 -3.933030605316162 -3.990032196044922 -3.9900319576263428
13.870500564575195 -2.842881441116333 -4.034221649169922 -4.291363716125488 -4.291363716125488
13.93850040435791 -2.859286308288574 -4.0166497230529785 -4.84518575668335 -4.845185279846191
13.577750205993652 -2.559265375137329 -3.7691967487335205 -4.843659400939941 -4.843659400939941
13.561750411987305 -2.3038909435272217 -3.5361897945404053 -3.0413765907287598 -3.0413761138916016
13.548750877380371 -2.2351064682006836 -3.491987466812134 -4.986218452453613 -4.9862189292907715
13.506250381469727 -2.2930846214294434 -3.539914846420288 -3.8450584411621094 -3.8450584411621094
13.494250297546387 -2.3561685085296

12.435250282287598 -1.062758445739746 -2.0913589000701904 -2.5502090454101562 -2.5502090454101562
12.464250564575195 -1.074554204940796 -2.077162981033325 -4.471905708312988 -4.4719061851501465
12.390000343322754 -1.0813509225845337 -2.076713800430298 -2.451798439025879 -2.4517982006073
12.415250778198242 -1.0753943920135498 -2.084163188934326 -0.7384248971939087 -0.7384252548217773
12.45875072479248 -1.0887930393218994 -2.0944533348083496 -4.13654899597168 -4.136548042297363
12.443500518798828 -1.064304232597351 -2.0754849910736084 -1.1935179233551025 -1.193518042564392
12.41200065612793 -1.0583419799804688 -2.073906660079956 -0.7405005097389221 -0.7405006289482117
12.422250747680664 -1.0783518552780151 -2.087117910385132 -2.6667556762695312 -2.6667556762695312
12.380500793457031 -1.0759172439575195 -2.0729691982269287 -1.6004250049591064 -1.6004245281219482
12.371500968933105 -1.0597063302993774 -2.073707103729248 -0.6730729341506958 -0.6730731725692749
12.453500747680664 -1.04260730

12.226250648498535 -0.9982518553733826 -1.9568955898284912 -4.8918962478637695 -4.891895771026611
12.266250610351562 -0.9808475971221924 -1.8944668769836426 -3.8621296882629395 -3.8621301651000977
12.293500900268555 -0.9959351420402527 -1.9172013998031616 -1.8027045726776123 -1.8027043342590332
12.257750511169434 -0.9849106669425964 -1.9006178379058838 -3.5305323600769043 -3.530531644821167
12.228750228881836 -0.9581815004348755 -1.8772088289260864 -2.3800883293151855 -2.3800880908966064
12.212250709533691 -0.9503980278968811 -1.8758248090744019 -5.012540340423584 -5.012540340423584
12.267251014709473 -0.964928388595581 -1.8682626485824585 -1.1000688076019287 -1.1000690460205078
12.246000289916992 -0.9606379866600037 -1.8530665636062622 -0.15067245066165924 -0.15067221224308014
12.273000717163086 -0.9658498167991638 -1.878006935119629 -3.3885817527770996 -3.3885817527770996
12.263500213623047 -0.9716372489929199 -1.8962801694869995 -1.7110698223114014 -1.711069941520691
12.178750991821

12.143000602722168 -0.9380383491516113 -1.817003607749939 -1.3119020462036133 -1.3119020462036133
12.226000785827637 -0.9304801225662231 -1.8227356672286987 -0.6457904577255249 -0.6457904577255249
12.173250198364258 -0.9251846075057983 -1.805742621421814 -2.2465639114379883 -2.2465641498565674
12.170750617980957 -0.9156430959701538 -1.790094256401062 -0.8967440128326416 -0.8967437744140625
12.163000106811523 -0.8997973203659058 -1.7638556957244873 -3.508533239364624 -3.508533477783203
12.216750144958496 -0.9174627065658569 -1.8150520324707031 -0.35758423805236816 -0.35758423805236816
12.142500877380371 -0.9327393770217896 -1.8169987201690674 -0.7128317952156067 -0.7128322720527649
12.189250946044922 -0.9403926134109497 -1.8414924144744873 -1.1719157695770264 -1.171915888786316
12.181750297546387 -0.9193865060806274 -1.810404658317566 -3.615288019180298 -3.6152875423431396
12.160500526428223 -0.9344496130943298 -1.827349305152893 -2.0590200424194336 -2.0590195655822754
12.12475013732910

12.154500961303711 -0.8847115635871887 -1.7477957010269165 -0.39060938358306885 -0.39060938358306885
12.137250900268555 -0.8866803646087646 -1.7653908729553223 -0.06919527053833008 -0.06919527053833008
12.097500801086426 -0.8933451175689697 -1.7640267610549927 -0.1040579080581665 -0.1040574312210083
12.144250869750977 -0.9180123209953308 -1.8304451704025269 -0.44732144474983215 -0.4473213851451874
12.054750442504883 -0.9117748141288757 -1.8126559257507324 -3.628032684326172 -3.628032684326172
12.13325023651123 -0.9312167167663574 -1.8201861381530762 -7.415753364562988 -7.415753364562988
12.149750709533691 -0.9272539615631104 -1.8145179748535156 -3.065387725830078 -3.065387725830078
12.104750633239746 -0.9145951867103577 -1.7781319618225098 -0.5990883111953735 -0.5990883708000183
12.16675090789795 -0.9081570506095886 -1.813944697380066 -0.2822704315185547 -0.28227055072784424
12.0600004196167 -0.8785910606384277 -1.7606881856918335 -1.981753945350647 -1.9817545413970947
12.0697507858276

...

11.547000885009766 -0.4698214828968048 -0.9516068696975708 -0.13258719444274902 -0.13258719444274902
11.575000762939453 -0.4741562008857727 -0.9545901417732239 -0.03422403335571289 -0.03422403335571289
11.617250442504883 -0.47902998328208923 -0.9694066047668457 -0.05677652359008789 -0.05677652359008789
11.63325023651123 -0.46335288882255554 -0.9589006304740906 -0.44438600540161133 -0.44438600540161133
11.562750816345215 -0.4647388458251953 -0.9654666185379028 -2.3101792335510254 -2.3101799488067627
11.625000953674316 -0.4714520275592804 -0.9760922193527222 -0.041390419006347656 -0.04138994216918945
11.614250183105469 -0.49461013078689575 -0.9852632284164429 -0.08230400085449219 -0.08230400085449219
11.623250961303711 -0.4879395365715027 -0.9818426370620728 -0.8124136924743652 -0.8124127388000488
11.616250991821289 -0.49709609150886536 -1.0102378129959106 -3.7148754596710205 -3.7148759365081787
11.588000297546387 -0.5023750066757202 -1.007087230682373 -0.07013940811157227 -0.07013940811

11.582250595092773 -0.4729488492012024 -0.9473479390144348 -0.14673638343811035 -0.14673638343811035
11.622750282287598 -0.46327871084213257 -0.9553555846214294 -0.47379541397094727 -0.47379493713378906
11.579500198364258 -0.46825191378593445 -0.9444477558135986 -1.1615686416625977 -1.1615684032440186
11.630000114440918 -0.4665282070636749 -0.9411762356758118 -1.3745397329330444 -1.3745396137237549
11.5910005569458 -0.464523583650589 -0.9242024421691895 -2.5152699947357178 -2.515270471572876
11.602500915527344 -0.4596669673919678 -0.9165073037147522 -0.8449345231056213 -0.8449345231056213
11.599750518798828 -0.4609559178352356 -0.9321525692939758 -1.2538542747497559 -1.2538554668426514
11.690250396728516 -0.4505140483379364 -0.9291051626205444 -0.15442955493927002 -0.15442943572998047
11.623250961303711 -0.47746342420578003 -0.9499139189720154 -0.10345792770385742 -0.10345792770385742
11.609750747680664 -0.46505409479141235 -0.9459239840507507 -0.1470508575439453 -0.1470499038696289
11

11.607500076293945 -0.4829771816730499 -0.9566965103149414 -0.0769038200378418 -0.0769038200378418
11.658750534057617 -0.48823946714401245 -0.9790642857551575 -2.5433454513549805 -2.543344736099243
11.631250381469727 -0.4790313243865967 -0.9641187191009521 -0.012565851211547852 -0.012565851211547852
11.542000770568848 -0.470897912979126 -0.9584815502166748 -0.12082481384277344 -0.12082481384277344
11.594500541687012 -0.47471773624420166 -0.9724031090736389 -1.758568525314331 -1.758568525314331
11.555000305175781 -0.4766707718372345 -0.9690567851066589 -0.3205941915512085 -0.3205941319465637
11.574000358581543 -0.46704959869384766 -0.9777051210403442 -1.9098174571990967 -1.9098188877105713
11.571250915527344 -0.4843420684337616 -0.969639241695404 -0.06321489810943604 -0.06321489810943604
11.609251022338867 -0.47168856859207153 -0.9657377004623413 -1.6815965175628662 -1.6815961599349976
11.53950023651123 -0.45735374093055725 -0.9431919455528259 -0.7537496089935303 -0.753749668598175
11.6

11.61400032043457 -0.4819865822792053 -0.979699969291687 -2.577934741973877 -2.577935218811035
11.585500717163086 -0.47165271639823914 -0.9564168453216553 -0.5892224311828613 -0.5892224311828613
11.623000144958496 -0.48206228017807007 -0.9864489436149597 -0.5475326776504517 -0.5475327968597412
11.61875057220459 -0.4783337712287903 -0.9567152261734009 -0.12048125267028809 -0.12048125267028809
11.589250564575195 -0.47631895542144775 -0.9657519459724426 -0.48065900802612305 -0.4806591272354126
11.667750358581543 -0.4769238233566284 -0.9888532757759094 -0.002174854278564453 -0.002174854278564453
11.58025074005127 -0.4804143011569977 -0.9806897044181824 -0.09944868087768555 -0.09944891929626465
11.60300064086914 -0.4874197542667389 -0.9954445362091064 -0.7238941192626953 -0.7238931655883789
11.600000381469727 -0.4770685136318207 -0.9813041090965271 -1.9003633260726929 -1.9003629684448242
11.631000518798828 -0.48747438192367554 -0.9975464344024658 -0.06745529174804688 -0.06745529174804688
11

11.57300090789795 -0.46650969982147217 -0.9576175808906555 -0.0015690326690673828 -0.0015690326690673828
11.659000396728516 -0.4843885600566864 -0.9602535963058472 -0.09636521339416504 -0.09636521339416504
11.578500747680664 -0.46952036023139954 -0.9550556540489197 -4.218033790588379 -4.218033790588379
11.614500999450684 -0.4616125524044037 -0.9407731294631958 -1.5644805431365967 -1.5644807815551758
11.649500846862793 -0.4758809506893158 -0.9479084610939026 -0.24571561813354492 -0.24571514129638672
11.612751007080078 -0.4702132046222687 -0.9564107060432434 -0.04206275939941406 -0.04206275939941406
11.584000587463379 -0.46320363879203796 -0.9457934498786926 -0.17671728134155273 -0.17671728134155273
11.649750709533691 -0.4744967520236969 -0.9565316438674927 -1.3659939765930176 -1.365993857383728
11.589750289916992 -0.4659349024295807 -0.9596955180168152 -1.3193697929382324 -1.3193702697753906
11.62125015258789 -0.4745140075683594 -0.9533692598342896 -0.10053801536560059 -0.10053801536560